In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport Compute_Jacobian_MA_journal
@pyimport GLS_Apr_weekend_journal

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in transpose(::Ptr{PyCall.PyObject_struct}) at ./deprecated.jl:770
 in transpose_f!(::Base.#transpose, ::Array{Ptr{PyCall.PyObject_struct},2}, ::Array{Ptr{PyCall.PyObject_struct},2}) at ./arraymath.jl:369
 in transpose(::Array{Ptr{PyCall.PyObject_struct},2}) at ./arraymath.jl:407
 in copy(::PyCall.PyArray{Ptr{PyCall.PyObject_struct},2}) at /home/jzh/.julia/v0.5/PyCall/src/numpy.jl:337
 in convert(::Type{Array{Ptr{PyCall.PyObject_struct},N}}, ::PyCall.PyObject) at /home/jzh/.julia/v0.5/PyCall/src/numpy.jl:453
 in convert(::Type{Array{PyCall.PyObject,N}}, ::PyCall.PyObject) at /home/jzh/.julia/v0.5/PyCall/src/numpy.jl:484
 in convert(::Type{PyCall.PyAny}, ::PyCall.PyObject) at /home/jzh/.julia/v0.5/PyCall/src/conversions.jl:806
 in #55 at ./<missing>:0 [inlined]
 in next at ./generator.jl:26 [inlined]
 in collect_to!(::Array{Expr,1}, ::Base.Generator{Array{Tuple{AbstractString,PyCall.PyObject},1},PyCall.##55#59}, ::Int64, ::Int64) at

In [2]:
numNodes = Compute_Jacobian_MA_journal.numNodes;
numLinks = Compute_Jacobian_MA_journal.numLinks;
numODpairs = Compute_Jacobian_MA_journal.numODpairs;
numZones = Compute_Jacobian_MA_journal.numZones
od_pairs = Compute_Jacobian_MA_journal.od_pairs;
link_list_js = Compute_Jacobian_MA_journal.link_list_js;
link_length_list = Compute_Jacobian_MA_journal.link_length_list;

flow_observ = GLS_Apr_weekend_journal.x_

258×30 Array{Float64,2}:
 1393.18     806.378    762.959        …  1122.05    1513.32     356.411 
 1245.77    1047.31    1023.88            1058.72    1286.13     737.719 
 2175.98    1580.96    1289.54            1310.68    1660.56    1011.89  
 2206.51    1358.68    1100.85            1375.95    2022.21     856.967 
   27.0447   279.236    279.358            114.702    182.177    319.842 
  143.919    260.583    207.122        …   112.767     47.7074    93.4537
  128.953     91.0833    77.1364           105.58     127.956     72.177 
  128.953     91.0833    77.1364           105.58     127.956     72.177 
 1381.89    1661.69    1435.11            1024.77    1282.9     1395.51  
 1061.96    1922.71    1849.59             893.572    782.416   2032.03  
   85.5516   284.675    357.062        …    50.566     29.8585   369.656 
  258.077    264.587    203.507            118.433    303.162    114.442 
  322.493    399.126    250.17             333.773    473.913    174.446 
    ⋮        

In [3]:
include("../Julia_files/initia_data.jl");
include("prepare_data_MA_journal.jl");
include("extract_data_MA_journal.jl");
include("../Julia_files/tap_MSA.jl");

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in readall(::String, ::Vararg{String,N}) at ./deprecated.jl:30
 in furInfo() at /home/jzh/Research/Traffic/InverseVIsTraffic/08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class_Apr_weekend_journal/prepare_data_MA_journal.jl:183
 in include_from_node1(::String) at ./loading.jl:488
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/jzh/.julia/v0.5/IJulia/src/execute_request.jl:169
 in eventloop(::ZMQ.Socket) at /home/jzh/.julia/v0.5/IJulia/src/eventloop.jl:8
 in (::IJulia.##9#15)() at ./task.jl:360
while loading /home/jzh/Research/Traffic/InverseVIsTraffic/08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class_Apr_weekend_journal/extract_data_MA_journal.jl, in expression starting on line 10
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in readall(::String, ::Vararg{String,N}) at ./deprecated.jl:30
 in furInfo() at /home/jzh/Research/Traffic/InverseVIsTra

In [4]:
deg = 6
c = 0.5
lam = 10000.0

coeffs_dict_Apr_weekend_ = readstring("../temp_files/coeffs_dict_Apr_weekend.json")
coeffs_dict_Apr_weekend_ = JSON.parse(coeffs_dict_Apr_weekend_)
fcoeffs = coeffs_dict_Apr_weekend_["($(deg),$(c),$(lam),1)"]
push!(fcoeffs, 0)
push!(fcoeffs, 0)

9-element Array{Any,1}:
  1.0       
  0.00268857
  0.0114664 
 -0.00834401
  0.0234205 
 -0.0137301 
  0.00429399
  0         
  0         

In [5]:
function socialObj(linkFlowVec)
    objVal = sum([free_flow_time[a] * fcoeffs[1] * linkFlowVec[a] + 
        free_flow_time[a] * fcoeffs[2] * linkFlowVec[a]^2 / capacity[a] + 
        free_flow_time[a] * fcoeffs[3] * linkFlowVec[a]^3 / capacity[a]^2 + 
        free_flow_time[a] * fcoeffs[4] * linkFlowVec[a]^4 / capacity[a]^3 + 
        free_flow_time[a] * fcoeffs[5] * linkFlowVec[a]^5 / capacity[a]^4 + 
        free_flow_time[a] * fcoeffs[6] * linkFlowVec[a]^6 / capacity[a]^5 + 
        free_flow_time[a] * fcoeffs[7] * linkFlowVec[a]^7 / capacity[a]^6 + 
        free_flow_time[a] * fcoeffs[8] * linkFlowVec[a]^8 / capacity[a]^7 + 
        free_flow_time[a] * fcoeffs[9] * linkFlowVec[a]^9 / capacity[a]^8 for a = 1:numLinks])
    return objVal
end

socialObj (generic function with 1 method)

In [6]:
weekend_Apr_list = [1, 7, 8, 14, 15, 21, 22, 28, 29]

9-element Array{Int64,1}:
  1
  7
  8
 14
 15
 21
 22
 28
 29

In [7]:
PoA_dict = Dict();
tapSocialFlowDicDict = Dict();
tapSocialFlowVecDict = Dict();

In [8]:
# demandsDict = readstring("../temp_files/demands_MA_Apr_weekend_journal.json");
# demandsDict = JSON.parse(demandsDict)

# demandsDict_ = Dict()
# for key in keys(demandsDict)
#     key_ = (parse(Int, split(split(key, ',')[1], '(')[2]), parse(Int, split(split(key, ',')[2], ')')[1]))
#     demandsDict_[key_] = demandsDict[key]
# end

# AA, BB = tapMSASocial(demandsDict_, fcoeffs);

In [9]:
for day in weekend_Apr_list

    demandsDict = readstring("../08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class_Apr_weekend_journal/results/demandsDictFixed$(day)_journal.json");
    demandsDict = JSON.parse(demandsDict)

    demandsDict_ = Dict()
    for key in keys(demandsDict)
        key_ = (parse(Int, split(split(key, ',')[1], '(')[2]), parse(Int, split(split(key, ',')[2], ')')[1]))
        demandsDict_[key_] = demandsDict[key]
    end

#     demandsDict_

#     tapFlowDicDict = Dict()
#     tapFlowVecDict = Dict()
#     tapFlowDicDict[day], tapFlowVecDict[day] = tapMSA(demandsDict_, fcoeffs);

#     tapFlowVecDict[day]

    tapSocialFlowDicDict[day], tapSocialFlowVecDict[day] = tapMSASocial(demandsDict_, fcoeffs);

#     tapSocialFlowVecDict[day]

#     flow_observ[:, day]

    # PoA_dict[day] = socialObj(tapFlowVecDict[day]) / socialObj(tapSocialFlowVecDict[day])

    PoA_dict[day] = socialObj(flow_observ[:, day]) / socialObj(tapSocialFlowVecDict[day]);
end

In [10]:
PoA_dict

Dict{Any,Any} with 9 entries:
  7  => 1.41419
  22 => 1.91873
  14 => 1.24442
  29 => 1.76935
  28 => 1.36469
  8  => 2.00178
  15 => 1.74115
  21 => 1.37647
  1  => 1.78943

In [11]:
tapSocialFlowVecDict

Dict{Any,Any} with 9 entries:
  7  => [802.283,682.013,1002.52,1088.85,0.0,0.0,819.616,891.126,2019.33,1537.9…
  22 => [788.05,665.524,993.862,1069.09,0.0,0.0,806.909,872.725,1979.45,1481.2 …
  14 => [802.283,682.013,1002.52,1088.85,0.0,0.0,819.616,891.126,2019.33,1537.9…
  29 => [787.834,667.754,997.969,1073.59,0.0,0.0,804.61,871.109,1976.29,1481.83…
  28 => [787.834,667.754,997.969,1073.59,0.0,0.0,804.61,871.109,1976.29,1481.83…
  8  => [802.283,682.013,1002.52,1088.85,0.0,0.0,819.616,891.126,2019.33,1537.9…
  15 => [802.283,682.013,1002.52,1088.85,0.0,0.0,819.616,891.126,2019.33,1537.9…
  21 => [788.05,665.524,993.862,1069.09,0.0,0.0,806.909,872.725,1979.45,1481.2 …
  1  => [802.283,682.013,1002.52,1088.85,0.0,0.0,819.616,891.126,2019.33,1537.9…

In [12]:
outfile = open("../temp_files/PoA_dict_Apr_weekend_journal.json", "w")

JSON.print(outfile, PoA_dict)

close(outfile)

In [13]:
outfile = open("../temp_files/tapSocialFlowVecDict_Apr_weekend_journal.json", "w")

JSON.print(outfile, tapSocialFlowVecDict)

close(outfile)